In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install transformers

import sys
sys.path.append('/content/drive/My Drive/colab_files/modules')

import infersent_glove_context_generation as ig

import time
import os
import contextlib
import torch
import nltk
nltk.download('punkt')

from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch

In [3]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', return_token_type_ids = True)
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')

In [4]:
def get_answer(url, question):
    context = ig.generate_context(url, question)
    print("\nContext: ", context)
    encoding = tokenizer.encode_plus(question, context)

    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

    start_scores, end_scores = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))

    ans_tokens = input_ids[torch.argmax(start_scores) : torch.argmax(end_scores)+1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens , skip_special_tokens=True)

    print ("\nQuestion: ", question)
    print ("\nAnswer Tokens: ", answer_tokens)

    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    print("\nAnswer: ", answer_tokens_to_string)
    return answer_tokens_to_string

In [5]:
get_answer('https://en.wikipedia.org/wiki/India', 'Which are the major Himalayan rivers in India?')

Vocab size : 100000
Nb words kept : 7300/11310 (64.5%)
Speed : 15.2 sentences/s (cpu mode, bsize=128)
Nb words kept : 5/10 (50.0%)
Speed : 25.7 sentences/s (cpu mode, bsize=128)

Context:  India has traditionally been the dominant country at the South Asian Games. India is notable for its religious diversity, with Hinduism, Buddhism, Sikhism, Islam, Christianity, and Jainism among the nation's major religions. They were imitated all over India and led to both the resurgence of Hinduism and the development of all modern languages of the subcontinent. In the last millennium, the most significant import of cooking techniques into India occurred during the Mughal Empire. Major Himalayan-origin rivers that substantially flow through India include the Ganges and the Brahmaputra, both of which drain into the Bay of Bengal. India (Hindi: Bhārat), officially the Republic of India (Hindi: Bhārat Gaṇarājya), is a country in South Asia. All states, as well as the union territories of Jammu and Kas

'ganges and the brahmaputra'

**FLASK**

In [6]:
%%capture
!pip install flask-ngrok

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

app = Flask(__name__, template_folder='drive/My Drive/colab_files/Flask/templates')
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

@app.route('/handle_data', methods=['GET', 'POST'])
def handle_data():
    url = request.form['inputURL']
    question = request.form['inputQuestion']
    answer = get_answer(url, question)
    print('\nAnswer: ', answer)
    return render_template('handle_data.html', result_url=url, result_question=question,
                           result_answer=answer)

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://95fd17039306.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [14/Oct/2020 12:52:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2020 12:52:39] "GET /favicon.ico HTTP/1.1" 404 -


Vocab size : 100000
Nb words kept : 3373/5002 (67.4%)


...

/content/drive/My Drive/colab_files/modules/models_infersent.py:197: UserWarning: No words in "['<s>', '</s>']" (idx=360) have w2v vectors.                                Replacing by "</s>"..
  Replacing by "</s>"..' % (sentences[i], i))


Speed : 15.1 sentences/s (cpu mode, bsize=128)
Nb words kept : 4/8 (50.0%)
Speed : 31.4 sentences/s (cpu mode, bsize=128)

Context:  Yoga is therefore seen as a process of interiorization or ascent of consciousness. Suggested origins are the Indus Valley Civilization (3300–1900 BCE) and pre-Vedic Eastern states of India, the Vedic period (1500–500 BCE), and the śramaṇa movement. It is the earliest literary work that highlights the fundamentals of yoga. These principles are significant to the history of spiritual side of yoga. One of the earliest and most influential sub-traditions of Vedanta, is Advaita Vedanta, which posits nondualistic monism. Theosophists including Madame Blavatsky also had a large influence on the Western public's view of Yoga. There is a broad variety of yoga schools, practices, and goals in Hinduism, Buddhism, and Jainism. Yoga is discussed in the ancient foundational Sutras of Hindu philosophy. Yoga is mentioned in the Rigveda,[note 1] and also referenced in the

127.0.0.1 - - [14/Oct/2020 13:30:00] "POST /handle_data HTTP/1.1" 200 -



Question:  What is the meaning of Yoga?

Answer Tokens:  ['interior', '##ization', 'or', 'ascent', 'of', 'consciousness']

Answer:  interiorization or ascent of consciousness

Answer:  interiorization or ascent of consciousness


127.0.0.1 - - [14/Oct/2020 13:38:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Oct/2020 13:38:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [14/Oct/2020 13:38:34] "GET / HTTP/1.1" 200 -
